# RouterChains

In [4]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains.llm import LLMChain

from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [5]:
fis_template = ChatPromptTemplate.from_template("""Você é um professor de física muito inteligente.
Você é ótimo em responder perguntas sobre física de forma concisa
e fácil de entender.
Quando você não sabe a resposta para uma pergunta, você admite
que não sabe.

Aqui está uma pergunta: {input}""")

mat_template = ChatPromptTemplate.from_template("""Você é um matemático muito bom.
Você é ótimo em responder perguntas de matemática.
Você é tão bom porque consegue decompor
problemas difíceis em suas partes componentes, 
responder às partes componentes e depois juntá-las
para responder à pergunta mais ampla.

Aqui está uma pergunta: {input}""")

hist_template = ChatPromptTemplate.from_template("""Você é um historiador muito bom.
Você tem um excelente conhecimento e compreensão de pessoas,
eventos e contextos de uma variedade de períodos históricos.
Você tem a capacidade de pensar, refletir, debater, discutir e
avaliar o passado. Você tem respeito pela evidência histórica
e a capacidade de usá-la para apoiar suas explicações
e julgamentos.

Aqui está uma pergunta: {input}""")


In [6]:
prompt_infos = [
    {'name': 'Fisica',
     'description': 'Ideal para responder perguntas sobre física',
     'prompt_template': fis_template},
    {'name': 'Matematica',
     'description': 'Ideal para responder perguntas sobre matemática',
     'prompt_template': mat_template},
    {'name': 'Historia',
     'description': 'Ideal para responder perguntas sobre história',
     'prompt_template': hist_template},
]

In [7]:
chat = ChatOpenAI(model='gpt-3.5-turbo-0125')

In [8]:
chains_destino = {}
for info in prompt_infos:
    chain = LLMChain(llm=chat, prompt=info['prompt_template'], verbose=True)
    chains_destino[info['name']] = chain

chains_destino

{'Fisica': LLMChain(verbose=True, prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='Você é um professor de física muito inteligente.\nVocê é ótimo em responder perguntas sobre física de forma concisa\ne fácil de entender.\nQuando você não sabe a resposta para uma pergunta, você admite\nque não sabe.\n\nAqui está uma pergunta: {input}'))]), llm=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001723F670650>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001723F0448D0>, model_name='gpt-3.5-turbo-0125', openai_api_key=SecretStr('**********'), openai_proxy='')),
 'Matematica': LLMChain(verbose=True, prompt=ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='Você é um matemático muito bom.\nVocê é ótimo em responder perguntas de matemática.\

In [10]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [12]:
prompt_infos

[{'name': 'Fisica',
  'description': 'Ideal para responder perguntas sobre física',
  'prompt_template': ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='Você é um professor de física muito inteligente.\nVocê é ótimo em responder perguntas sobre física de forma concisa\ne fácil de entender.\nQuando você não sabe a resposta para uma pergunta, você admite\nque não sabe.\n\nAqui está uma pergunta: {input}'))])},
 {'name': 'Matematica',
  'description': 'Ideal para responder perguntas sobre matemática',
  'prompt_template': ChatPromptTemplate(input_variables=['input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='Você é um matemático muito bom.\nVocê é ótimo em responder perguntas de matemática.\nVocê é tão bom porque consegue decompor\nproblemas difíceis em suas partes componentes, \nresponder às partes componentes e depois juntá-las\npara respond

In [15]:
destinos = [f'{p["name"]}: {p["description"]}' for p in prompt_infos]
destinos_str = '\n'.join(destinos)
print(destinos_str)

Fisica: Ideal para responder perguntas sobre física
Matematica: Ideal para responder perguntas sobre matemática
Historia: Ideal para responder perguntas sobre história


In [17]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinos_str
)
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
Fisica: Ideal para responder perguntas sobre física
Matematica: Ideal para responder perguntas sobre 

In [18]:
router_template = PromptTemplate(
    template=router_template,
    input_variables=['input'],
    output_parser=RouterOutputParser()
)

router_chain = LLMRouterChain.from_llm(chat, router_template, verbose=True)

In [19]:
default_prompt = ChatPromptTemplate.from_template('{input}')
default_chain = LLMChain(llm=chat, prompt=default_prompt, verbose=True)

chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=chains_destino,
    default_chain=default_chain,
    verbose=True)

In [20]:
chain.invoke({'input': 'O que é um buraco negro?'})



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
Fisica: {'input': 'O que é um buraco negro?'}

> Entering new LLMChain chain...
Prompt after formatting:
Human: Você é um professor de física muito inteligente.
Você é ótimo em responder perguntas sobre física de forma concisa
e fácil de entender.
Quando você não sabe a resposta para uma pergunta, você admite
que não sabe.

Aqui está uma pergunta: O que é um buraco negro?

> Finished chain.

> Finished chain.


{'input': 'O que é um buraco negro?',
 'text': 'Um buraco negro é uma região do espaço onde a gravidade é tão intensa que nada, nem mesmo a luz, pode escapar de seu campo gravitacional. Essa intensa gravidade é causada pela alta concentração de massa em um espaço muito pequeno. Quando uma estrela muito massiva colapsa sob sua própria gravidade, pode formar um buraco negro. Essa região é rodeada por um limite chamado de horizonte de eventos, que marca o ponto de onde a luz não pode mais escapar. Os buracos negros são objetos fascinantes e desafiadores para os cientistas estudarem.'}

In [21]:
chain.invoke({'input': 'O que é uma equação quadrática?'})



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
Matematica: {'input': 'O que é uma equação quadrática?'}

> Entering new LLMChain chain...
Prompt after formatting:
Human: Você é um matemático muito bom.
Você é ótimo em responder perguntas de matemática.
Você é tão bom porque consegue decompor
problemas difíceis em suas partes componentes, 
responder às partes componentes e depois juntá-las
para responder à pergunta mais ampla.

Aqui está uma pergunta: O que é uma equação quadrática?

> Finished chain.

> Finished chain.


{'input': 'O que é uma equação quadrática?',
 'text': 'Uma equação quadrática é uma equação polinomial de segundo grau, ou seja, uma equação na forma ax^2 + bx + c = 0, onde a, b e c são constantes e a ≠ 0. A incógnita x representa a variável da equação e as soluções da equação são os valores de x que satisfazem a igualdade. As equações quadráticas podem ter zero, uma ou duas soluções reais, dependendo do valor do discriminante (b^2 - 4ac). As soluções podem ser encontradas através da fórmula de Bhaskara ou completando o quadrado, por exemplo.'}

In [22]:
chain.invoke({'input': 'Quando foi a revolução industrial?'})



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
Historia: {'input': 'Quando ocorreu a revolução industrial?'}

> Entering new LLMChain chain...
Prompt after formatting:
Human: Você é um historiador muito bom.
Você tem um excelente conhecimento e compreensão de pessoas,
eventos e contextos de uma variedade de períodos históricos.
Você tem a capacidade de pensar, refletir, debater, discutir e
avaliar o passado. Você tem respeito pela evidência histórica
e a capacidade de usá-la para apoiar suas explicações
e julgamentos.

Aqui está uma pergunta: Quando ocorreu a revolução industrial?

> Finished chain.

> Finished chain.


{'input': 'Quando ocorreu a revolução industrial?',
 'text': 'A Revolução Industrial ocorreu principalmente no século XVIII, com início na Grã-Bretanha por volta da segunda metade do século. Esse período de transformações profundas na produção, tecnologia e organização do trabalho teve impacto significativo em todo o mundo, marcando o início da era moderna.'}